### As of 6.6.24 this is not curated and some adjustments need to be done

In [ ]:
from utils.data_utils import split_dataset, TimeSeriesDataset
from utils.evaluation_utils import plot_multistep_forecast, evaluate_on_test_data
from utils.training_utils import train
import pandas as pd 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch

ETTm2 = pd.read_csv("/workspaces/time_series_experiment/ETT-small/ETTm2.csv")
training_df, val_df, test_df = split_dataset(ETTm2, remain_same = False)

p_lag = 96
future_steps = round(1)
batch_size = 8
epochs = 5
learning_rate=1.e-4
decomp_kernel_size = 24
number_of_forecasts = 100
target_column = ['OT']
feature_columns = ['OT'] #[i for i in training_df.columns]
modelling_task = 'univariate'

net = train(
            epochs = epochs, 
            n_continous_features=1, 
            n_categorial_features=0,
            p_lag=  p_lag, 
            future_steps = future_steps, 
            training_df = training_df, 
            validation_df = val_df, 
            feature_columns = feature_columns,
            target_column = target_column, 
            learning_rate=learning_rate ,
            decomp_kernel_size= decomp_kernel_size, 
            batch_size=batch_size, 
            model = 'dlinear', 
            modelling_task = modelling_task, 
            density=True
            )

steps = 1000
batch_size = 8
p_lag = 96

test_data = DataLoader(TimeSeriesDataset(test_df,future_steps= future_steps, target_column = target_column,feature_columns=feature_columns,p_lag=p_lag, modelling_task='univariate'), batch_size=batch_size,drop_last=True)      

target_l = []
for i, (inputs, targets) in enumerate(test_data): 
    if i > round(steps/batch_size): 
        break
    else: 
        [target_l.append(i.item()) for i in targets.reshape(batch_size)]
target_l = target_l[0:steps]

output_l = []
for i in range(steps): 
    if i == 0: 
        dist = net(inputs.squeeze(1))         
        sample = dist.sample()
        new_input = torch.cat((inputs[:,:,1:p_lag], sample.reshape(batch_size,1,1)),2)
        [output_l.append([i.item()]) for i in sample.reshape(batch_size)]
    else: 
        dist = net(new_input.squeeze(1))         
        sample = dist.sample()
        new_input = torch.cat((inputs[:,:,1:p_lag], sample.reshape(batch_size,1,1)),2)
        for u in range(batch_size): 
            item = sample.reshape(batch_size)[u].item()
            output_l[u].append(item)

plt.plot(range(0, len(target_l)), target_l, 'g', label='target time series', alpha=1)
for output in output_l: 
    plt.plot(range(0, len(target_l)), output, color='#F39C12',linewidth=1, linestyle='-.',alpha=0.4, label='pred time series' + "\n" + f'{steps} steps')